In [132]:
import torch
from torch import nn, optim
import torch.utils.data as torch_data
import torch.nn.functional as F
import itertools as it
import numpy as np
import random
from itertools import combinations, product
from mutex import EncDec, Vocab, batch_seqs, Mutex
from data import encode,  generate_fig2_exp, Oracle, collate, eval_format
from absl import app, flags
import sys
import os
from main import pretrain, train, validate
import hlog
import pickle

In [133]:
import hlog

import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['figure.dpi'] = 250

#input_symbols_list   = set(['dax', 'lug', 'wif', 'zup', 'fep', 'blicket', 'kiki', 'tufa', 'gazzer'])
input_symbols_list   = ['pink', 'yellow', 'purple', 'red', 'thrice', 'gren', 'blue', 'around', 'after']
output_symbols_list  = ['RED', 'YELLOW', 'BLUE', 'GREEN', 'PURPLE', 'PINK']

DEVICE = torch.device("cuda:0")

In [185]:
FLAGS = flags.FLAGS
delattr(FLAGS,"debug")
delattr(FLAGS,"seed")
delattr(FLAGS,"full_data")
flags.DEFINE_bool("debug", True, "debug mode")
flags.DEFINE_integer("seed", 1, "random seed")
flags.DEFINE_bool("full_data", True, "full figure 2 experiments, otherwise color matching")
FLAGS(['mutex.ipynb'])

['mutex.ipynb']

In [186]:
random.seed(FLAGS.seed)
np.random.seed(FLAGS.seed)
torch.manual_seed(FLAGS.seed)
model = torch.load(f"exp/full2/seed_{FLAGS.seed}_model.m")
model.eval()
train_items, test_items = model.py.train_items, model.py.test_items
vocab_x, vocab_y = model.vocab_x, model.vocab_y

In [187]:
FLAGS.debug = False

In [188]:
validate(model, train_items, vis=True)

(1.0, 1.0)

In [189]:
validate(model, test_items, vis=True)

(0.36363636363636365, 0.972972972972973)

In [192]:
ys = model.py.sample(1,7)

In [193]:
model.vocab_y.decode(ys.detach().cpu().transpose(0,1).tolist()[0])

['<s>', 'RED', 'RED', 'RED', 'RED', 'RED', '</s>']

In [213]:
samples = []
for i in range(100):
    xs, _ = model.qxy.sample(ys, model.MAXLEN_X, temp=model.temp)
    ux = [list(x) for x in set(tuple(x) for x in xs)]
    samples += [model.vocab_x.decode(ux[0])]
    xps   = batch_seqs(ux).to(DEVICE)

In [218]:
ux = [list(x) for x in set(tuple(x) for x in samples)]

In [220]:
ux 

[['<s>', 'red', 'thrice', 'around', 'red', 'after', 'after', '</s>'],
 ['<s>', 'red', 'around', 'after', 'red', 'red', 'red', '</s>'],
 ['<s>', 'red', 'thrice', 'around', 'around', 'red', 'green', '</s>'],
 ['<s>', 'red', 'thrice', 'around', 'around', 'green', 'purple', '</s>'],
 ['<s>', 'red', 'green', 'red', 'around', 'thrice', 'yellow', '</s>'],
 ['<s>', 'red', 'thrice', 'thrice', 'around', 'purple', 'red', '</s>'],
 ['<s>', 'red', 'after', 'red', 'red', 'around', 'red', '</s>'],
 ['<s>', 'red', 'thrice', 'thrice', 'around', 'red', 'red', '</s>'],
 ['<s>', 'red', 'red', 'red', 'around', 'after', 'thrice', '</s>'],
 ['<s>', 'red', 'thrice', 'around', 'red', 'thrice', 'blue', '</s>'],
 ['<s>', 'red', 'thrice', 'green', 'red', 'around', 'around', '</s>'],
 ['<s>', 'red', 'thrice', 'around', 'after', 'thrice', 'around', '</s>'],
 ['<s>', 'red', 'around', 'red', 'around', 'after', 'after', '</s>'],
 ['<s>', 'red', 'thrice', 'around', 'after', 'red', '</s>'],
 ['<s>', 'red', 'around', 'th

In [126]:
for y in ys.split(1,dim=1):
    ybatch = y.repeat(1, xps.shape[1])
    logprob_qxy = model.qxy.logprob(ys, xps)
    print(logprob_qxy)

tensor([-5.6818], device='cuda:0', grad_fn=<NegBackward>)


In [122]:
logp.argmax(dim=2)

tensor([[12],
        [12],
        [12],
        [12],
        [11],
        [11],
        [ 2]], device='cuda:0')

In [127]:
F.nll_loss(logp.permute(1,2,0), xps[1:,:].transpose(0,1), reduction='none')

ValueError: Expected target size (1, 7), got torch.Size([1, 12])

In [128]:
xps[1:,:].transpose(0,1)

tensor([[ 6,  9,  6, 13,  6,  8,  2,  0,  0,  0,  0,  0]], device='cuda:0')

In [129]:
logprob_qxy

tensor([-5.6818], device='cuda:0', grad_fn=<NegBackward>)

In [130]:
logprob_px = model.px.logprob(xps)

In [131]:
logprob_px 

tensor([-26.6667], device='cuda:0', grad_fn=<NegBackward>)